In [ ]:
!pip install yfinance
import pandas as pd 
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

     |████████████████████████████████| 5.5MB 11.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=94c7ed27ba59a3e206394cb4bc7b476c4c25c25d61d52720f9d058ce14d43f37
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
data = yf.download('SBIN.NS', 
                      start='2020-01-15', 
                      end='2021-04-15',
                      progress=False)

In [ ]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-04-07,351.250000,363.200012,347.600006,358.549988,358.549988,48023602
2021-04-08,361.000000,363.299988,354.299988,355.600006,355.600006,33370259
2021-04-09,354.399994,364.500000,351.200012,353.000000,353.000000,46473100
2021-04-12,344.000000,344.000000,322.549988,328.850006,328.850006,75501713
2021-04-13,332.000000,342.250000,330.500000,341.000000,341.000000,49773360


In [ ]:
def checker(stock_name,start_date,end_date,Amount):
  data = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    progress=False)
  # pct change
  data['Pre Pct Change']=(data['Close'].pct_change() * 100).shift(1)
  # # pre close
  # data['Pre Close']=data['Close'].shift(1)
  # pre high
  data['Pre High']=data['High'].shift(1)
  # color
  data['Color']=['Red' if x[0]>x[1] else 'Green' for x in zip(data['Open'],data['Close'])]
  # gapup
  data['Gap']=['Gap up' if data['Open'][i] > data['High'][i-1] else 'Down' for i in range(0,len(data))]
  # trigger
  try:
    data['Trigered Buy'] = ['Yes' if data['High'][i-1] > data['Low'][i] else 'No' for i in range(0,len(data))]
  except:
    pass
  # dlt column
  data=data.drop(['Adj Close','Volume'],axis=1)
  # success
  data['Success']= ['Yes' if data['High'][i-1] > data['Close'][i] else 'No' for i in range(0,len(data))]
  # drop na and reset index
  data=data.dropna()
  data=data.reset_index()
  # slice 
  data=data[data['Pre Pct Change'] > 5]
  data=data[data['Gap'] == 'Gap up']
  data=data[data['Trigered Buy'] == 'Yes']
  # profit
  data['P/L']= data['Pre High'] - data['Close']
  # add stoploss conditions
  data=data.reset_index().drop('index',axis=1)
  data['P/L']=[data['P/L'][i] if data['Success'][i]=='Yes' else data['Pre High'][i] - data['Open'][i] for i in range(0,len(data))]
  # amount calculation
  data['INR'] = (round(Amount/data['Pre High'])) * data['P/L']
  # return
  data['Date']=data['Date'].astype('str')
  return sum(data['INR']), len(data),list(data['Date']) 


In [ ]:
from tqdm import tqdm
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd '/content/gdrive/MyDrive/SASH/'
nse=pd.read_csv('ALL_FNO.csv')

/content/gdrive/MyDrive/SASH


In [ ]:
nse.head(2)

,Stock Name,Lot Size,Symbol
0,apollo hospitals enterpirse,500,APOLLOHOSP.NS
1,acc limited,500,ACC.NS


In [ ]:
checker('ADANIPORTS.NS','2021-04-13','2021-04-16',100000)

(0, 0, [])

In [ ]:
bull_sash_stocks={}
for i in tqdm(nse['Symbol']):
    bull_sash_stocks[i]=checker(i,'2021-02-01','2021-03-01',100000)
final_bull_sash=pd.DataFrame.from_dict(bull_sash_stocks,orient='index',columns=['P/L','Counts','Dates'],)
final_bull_sash.reset_index(inplace=True)
final_bull_sash=final_bull_sash.rename(columns={'index':'Ticker'})
# final_bull_sash.to_csv('bull_gapup_F16_nostoploss.csv')
final_bull_sash=final_bull_sash[final_bull_sash['Counts'] != 0]
print('\n')
print('Total INR:',sum(final_bull_sash['P/L']))
final_bull_sash=final_bull_sash.reset_index().drop('index',axis=1)
final_bull_sash


100%|██████████| 137/137 [00:21<00:00,  6.28it/s]



Total INR: 15037.991889953613


,Ticker,P/L,Counts,Dates
0,ADANIENT.NS,-23.089722,2,"[2021-02-03, 2021-02-15]"
1,APOLLOTYRE.NS,546.250000,1,[2021-02-03]
2,BANKBARODA.NS,721.798164,1,[2021-02-05]
3,EXIDEIND.NS,2443.301407,1,[2021-02-09]
4,GODREJPROP.NS,979.794800,1,[2021-02-03]
5,LICHSGFIN.NS,2100.801270,1,[2021-02-16]
6,NAUKRI.NS,1985.000000,1,[2021-02-03]
7,PEL.NS,912.601318,1,[2021-02-16]
8,RAMCOCEM.NS,888.098694,1,[2021-02-09]
9,TATAMOTORS.NS,-3146.401855,1,[2021-02-03]


In [ ]:
# once stoploss hit # no trading
# but stop loss was modified and got profit 1st time then take trade again # example mandm fin 30-3-21
# tata chem stop loss # loss 750
# tata motors waste
# dont modify smaller stop loss
# always buy after high break and same candle break by next candle, i.e next candle break 

In [ ]:
data = yf.download('ACC.NS', 
                      start='2020-01-01', 
                      end='2021-04-14',
                      progress=False)
# pct change
data['Pre Pct Change']=(data['Close'].pct_change() * 100).shift(1)
# # pre close
# data['Pre Close']=data['Close'].shift(1)
# pre high
data['Pre High']=data['High'].shift(1)
# color
data['Color']=['Red' if x[0]>x[1] else 'Green' for x in zip(data['Open'],data['Close'])]
# gapup
data['Gap']=['Gap up' if data['Open'][i] > data['High'][i-1] else 'Down' for i in range(0,len(data))]
# trigger
try:
  data['Trigered Buy'] = ['Yes' if data['High'][i-1] > data['Low'][i] else 'No' for i in range(0,len(data))]
except:
  pass
# dlt column
data=data.drop(['Adj Close','Volume'],axis=1)
# success
data['Success']= ['Yes' if data['High'][i-1] > data['Close'][i] else 'No' for i in range(0,len(data))]
# drop na and reset index
data=data.dropna()
data=data.reset_index()
# slice 
data=data[data['Pre Pct Change'] > 5]
data=data[data['Gap'] == 'Gap up']
data=data[data['Trigered Buy'] == 'Yes']
# profit
data['P/L']= data['Pre High'] - data['Close']
data=data.reset_index().drop('index',axis=1)
data['P/L']=[data['P/L'][i] if data['Success'][i]=='Yes' else data['Pre High'][i] - data['Open'][i] for i in range(0,len(data))]

data['INR'] = (round(100000/data['Pre High'])) * data['P/L']
# return

data

,Date,Open,High,Low,Close,Pre Pct Change,Pre High,Color,Gap,Trigered Buy,Success,P/L,INR
0,2020-03-27,1000.0,1010.0,940.000000,959.150024,5.211280,980.000000,Red,Gap up,Yes,Yes,20.849976,2126.697510
1,2021-02-02,1700.0,1784.0,1691.050049,1762.849976,5.087756,1696.949951,Green,Gap up,Yes,No,-3.050049,-179.952881


['2020-03-27', '2021-02-02']